In [1]:
!cat inference_model/code/inference.py

DEBUG = False # false for deployment
from PIL import Image

print('PIL imported')

import numpy as np
import os
import json

import sys
if DEBUG:
    sys.path.append('pytorch-image-models')
else:
    sys.path.append('inference_model/code')
    sys.path.append('inference_model/code/pytorch-image-models')
    sys.path.append('inference_model/code/boto3')

try:
    import boto3
except Exception as e:
    print('boto3 failed')
    print('boto3 error', e)
else:
    print('boto3 imported')
    
try:
    import torchvision.transforms as transforms
except Exception as e:
    print('torchvision failed')
    print('torchvision error', e)
else:
    print('torchvision imported')

if DEBUG:
    print('DEBUG MODE')
else:
    print('pwd:', os.getcwd())
    print('ls ', os.listdir())
    print('ls code', os.listdir('code'))
    print('ls inference_model', os.listdir('inference_model'))
    print('ls inference_model/code', os.listdir('inference_model/code'))

import torch
import timm
import math
from t

In [2]:
!tar -czvf model.tar.gz inference_model/code inference_model/model_state_dict.pth

inference_model/code/
inference_model/code/python_dateutil-2.8.2.dist-info/
inference_model/code/python_dateutil-2.8.2.dist-info/WHEEL
inference_model/code/python_dateutil-2.8.2.dist-info/METADATA
inference_model/code/python_dateutil-2.8.2.dist-info/LICENSE
inference_model/code/python_dateutil-2.8.2.dist-info/top_level.txt
inference_model/code/python_dateutil-2.8.2.dist-info/zip-safe
inference_model/code/python_dateutil-2.8.2.dist-info/RECORD
inference_model/code/python_dateutil-2.8.2.dist-info/INSTALLER
inference_model/code/urllib3-1.26.9.dist-info/
inference_model/code/urllib3-1.26.9.dist-info/WHEEL
inference_model/code/urllib3-1.26.9.dist-info/METADATA
inference_model/code/urllib3-1.26.9.dist-info/top_level.txt
inference_model/code/urllib3-1.26.9.dist-info/LICENSE.txt
inference_model/code/urllib3-1.26.9.dist-info/RECORD
inference_model/code/urllib3-1.26.9.dist-info/INSTALLER
inference_model/code/s3transfer/
inference_model/code/s3transfer/copies.py
inference_model/code/s3transfer/up

In [3]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [4]:
upload_file('model.tar.gz', 'project-ml-bucket')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


True

In [5]:
# from sagemaker import get_execution_role
# from sagemaker.pytorch.model import PyTorchModel
# role = get_execution_role()

# pytorch_model = PyTorchModel(model_data='s3://project-ml-bucket/inference_model.tar.gz', role=role,
#                              framework_version='1.10.0',
#                              py_version='py38',
#                              entry_point='inference_model/code/inference.py',
#                              env={'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})

In [6]:
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
role = get_execution_role()

pytorch_model = PyTorchModel(model_data='s3://project-ml-bucket/model.tar.gz', role=role,
                             framework_version='1.9.0',
                             py_version='py38',
                             entry_point='inference_model/code/inference.py',
                             env={'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})

In [7]:
pytorch_model

In [8]:
from sagemaker.serializers import JSONSerializer

predictor = pytorch_model.deploy(
    instance_type="ml.m4.xlarge", 
    initial_instance_count=1,
    serializer=JSONSerializer()
)

--------!

In [9]:
predictor.endpoint_name

'pytorch-inference-2022-04-28-05-15-48-115'

In [11]:
response = predictor.predict({"bucket": "project-ml-bucket", "key": "bottle_1_a.jpg"})

In [12]:
response

array([[ 0.94092983, -0.89762574, -0.12270916, ..., -0.74705303,
         0.20922968, -0.17917496]])

In [32]:
ls

Deploy.ipynb     inference_model/  model.tar.gz  toy_model/
DeployToy.ipynb  lost+found/       __pycache__/  toy_model.tar.gz


In [16]:
!pip install boto3 --target inference_model/code/

     |████████████████████████████████| 132 kB 24.0 MB/s            
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
     |████████████████████████████████| 8.7 MB 56.2 MB/s            
     |████████████████████████████████| 247 kB 68.3 MB/s            
     |████████████████████████████████| 138 kB 67.7 MB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.
awscli 1.22.80 requires botocore==1.24.25, but you have botocore 1.24.42 which is incompatible.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.24.42 which is incompatible.


In [15]:
!ls inference_model/code/

inference_notebook.ipynb  __pycache__		requirements.txt
inference.py		  pytorch-image-models


In [14]:
!rm -rf inference_model/code/torchvision-0.9.0.dist-info